In [1]:
import pickle
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import defaultdict
from datetime import datetime

from tqdm import tqdm
from scipy.stats import spearmanr, kendalltau
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import OneHotEncoder

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:

    взять в тренировочный набор турниры с dateStart из 2019 года; 

    в тестовый — турниры с dateStart из 2020 года.

In [2]:
def get_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
        return data

In [3]:
results = get_data('data/results.pkl')
players = get_data('data/players.pkl')
tournaments = get_data('data/tournaments.pkl')

In [4]:
def clear_data(results, tournaments):
    masked_results = defaultdict(list)
    for id_, teams in results.items():
        tournaments[id_]['year'] = datetime.fromisoformat(tournaments[id_]['dateStart']).year
        if  tournaments[id_]['year'] in {2019, 2020}:
            for team in teams:
                if team.get('mask', None) is not None and len(team.get('teamMembers', [])) > 0:
                    masked_results[id_].append(team)
    return masked_results, {id_: tournaments[id_] for id_ in masked_results}

In [5]:
results, tournaments = clear_data(results, tournaments)
len(results), len(tournaments)

(848, 848)

In [6]:
train_ids = {key for key, value in tournaments.items() if value['year'] == 2019}
test_ids = {key for key, value in tournaments.items() if value['year'] == 2020}

In [7]:
len(train_ids), len(test_ids)

(675, 173)

In [8]:
results_train = {idx: results[idx] for idx in train_ids}
results_test = {idx: results[idx] for idx in test_ids}

In [9]:
def get_dataframe(results, tournaments):
    data = defaultdict(list)

    for tournament_id, tournament in results.items():
        for result in tournament:
            if result.get('mask', None) is None:
                continue
            count_qiestions = sum(c for c in tournaments[tournament_id]['questionQty'].values())
            mask = result['mask'].replace('X', '').replace('?', '')
            if len(mask) > 60 or (len(mask) != count_qiestions and len(result['mask']) != count_qiestions):
                continue
            for question_id, m in enumerate(mask):
                for member in result['teamMembers']:
                    data['question_id'].append(question_id)
                    data['tournament_id'].append(tournament_id)
                    data['team_id'].append(result['team']['id'])
                    data['member_id'].append(member['player']['id'])
                    data['team_pos'].append(result['position'])
                    data['result'].append(int(m))

    return pd.DataFrame.from_dict(data)

In [10]:
df_train = get_dataframe(results_train, tournaments)
df_test = get_dataframe(results_test, tournaments)

In [11]:
df_train['tour_ques_id'] = df_train[['tournament_id', 'question_id']].apply(lambda x: '_'.join(map(str, x)), axis=1)
df_test['tour_ques_id'] = df_test[['tournament_id', 'question_id']].apply(lambda x: '_'.join(map(str, x)), axis=1)

In [12]:
df_train.head()

,question_id,tournament_id,team_id,member_id,team_pos,result,tour_ques_id
0,0,6144,27254,27469,1.0,1,6144_0
1,0,6144,27254,57286,1.0,1,6144_0
2,0,6144,27254,155103,1.0,1,6144_0
3,0,6144,27254,41104,1.0,1,6144_0
4,0,6144,27254,57288,1.0,1,6144_0


Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


In [13]:
def get_questions_complexity(df):
    questions_count = df.tour_ques_id.value_counts()
    questions_complex = {}
    for row in df.groupby(by='tour_ques_id'):
        quest_id = row[0]
        questions_complex[quest_id] = 1 - row[1]['result'].sum() / questions_count[quest_id]
    return questions_complex

In [14]:
questions_comp_train = get_questions_complexity(df_train)
questions_comp_test = get_questions_complexity(df_test)

In [15]:
len(questions_comp_train), len(questions_comp_test)

(22374, 5968)

In [16]:
df_train['complex'] = df_train['tour_ques_id'].apply(lambda x: questions_comp_train[x])
df_test['complex'] = df_test['tour_ques_id'].apply(lambda x: questions_comp_test[x])

In [140]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df_train[['member_id', 'tour_ques_id']])
X_train = encoder.transform(df_train[['member_id', 'tour_ques_id']])
X_test = encoder.transform(df_test[['member_id', 'tour_ques_id']])
y_train = df_train.result.values

In [181]:
model = LogisticRegression(n_jobs=-1)
model.fit(X_train, y_train)

LogisticRegression(n_jobs=-1)

In [182]:
predict_train = model.predict_proba(X_train)[:, 1]
predict_test = model.predict_proba(X_test)[:, 1]

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

In [143]:
df_test['neg_predict'] = 1 - predict_test

In [144]:
df_test.head()

,question_id,tournament_id,team_id,member_id,team_pos,result,tour_ques_id,complex,neg_predict,predict
0,0,6160,54613,36568,1.0,1,6160_0,0.632911,0.309796,0.279892
1,0,6160,54613,117262,1.0,1,6160_0,0.632911,0.509811,0.228755
2,0,6160,54613,25475,1.0,1,6160_0,0.632911,0.341398,0.224340
3,0,6160,54613,140120,1.0,1,6160_0,0.632911,0.548137,0.160483
4,0,6160,54613,14347,1.0,1,6160_0,0.632911,0.383099,0.209076


In [145]:
def compute_scores(df_test):
    kendall = []
    spearman = []
    for tournament_id in test_ids:
        tour_df = df_test[df_test['tournament_id']==tournament_id]
        kendall.append(kendalltau(tour_df['neg_predict'], tour_df['team_pos'])[0])
        spearman.append(spearmanr(tour_df['neg_predict'], tour_df['team_pos'])[0])
    kendall = np.array(kendall)[~np.isnan(kendall)]
    spearman = np.array(spearman)[~np.isnan(spearman)]
    return np.mean(kendall), np.mean(spearman)

In [146]:
best_kendall, best_spearman = compute_scores(df_test)
best_model = model
print(f'Kendall: {best_kendall} spearman: {best_spearman}')

Kendall: 0.511707310365948 spearman: 0.6706454769681249


Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
разработайте EM-схему для обучения этой модели, реализуйте её в коде;
обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


In [147]:
for step in range(1, 6):
    df_train['predict'] = predict_train
    df_train['neg_predict'] = 1 - predict_train
    predicts_by_teams = df_train.groupby(['tournament_id', 'team_id', 'tour_ques_id']).agg('prod').reset_index()
    predicts_by_teams['team_predict'] = 1 - predicts_by_teams['neg_predict']
    
    predicts_by_teams = pd.merge(df_train, predicts_by_teams.drop(columns=['neg_predict', 'predict']), on=['tournament_id', 'team_id', 'tour_ques_id'])
    z = predicts_by_teams['predict'] / predicts_by_teams['team_predict']
    z = np.where(y_train == 0, 0, z)
  
    model = LinearRegression(n_jobs=-1)
    model.fit(X_train, z)
    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)
  
    df_test['predict'] = predict_test    
    df_test['neg_predict'] = 1 - predict_test
    predicts_by_teams = df_test.groupby(['tournament_id', 'team_id', 'tour_ques_id']).agg('prod').reset_index()
    predicts_by_teams['team_predict'] = 1 - predicts_by_teams['neg_predict']
    
    predicts_by_teams = pd.merge(df_test, predicts_by_teams.drop(columns=['neg_predict', 'predict', 'team_pos']), on=['tournament_id', 'team_id', 'tour_ques_id'])
    kendall, spearman = compute_scores(predicts_by_teams)
    if best_kendall < kendall and best_spearman < spearman:
        best_kendall = kendall
        best_spearman = spearman
        best_model = model
    print(f'Step: {step} Kendall: {kendall} spearman: {spearman}')

Step: 1 Kendall: 0.5316876999145242 spearman: 0.6907979084557259
Step: 2 Kendall: 0.5184091395306106 spearman: 0.6736052235941389
Step: 3 Kendall: 0.4997577311158762 spearman: 0.6526956485324784
Step: 4 Kendall: 0.48899811723252795 spearman: 0.6398377761912305
Step: 5 Kendall: 0.4695304867620617 spearman: 0.6176975414748395


А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [200]:
questions_weight = best_model.coef_[df_train.member_id.nunique():]

In [219]:
tour_compl = defaultdict(list)

for question_weight, question_id in zip(questions_weight, df_train.tour_ques_id.unique()):
    tour_compl[int(question_id.split('_')[0])].append(question_weight)

In [220]:
for tournament_id, questions_weights in tour_compl.items():
    tour_compl[tournament_id] = np.mean(questions_weights)

In [221]:
tour_compl = sorted([(key, (val)) for key, val in tour_compl.items()], key=lambda x: x[1])

In [222]:
print('Турниры с самыми сложными вопросами:')
for idx in tour_compl[:10]:
    print(f"{idx[0]}. {tournaments[idx[0]]['name']}")

Турниры с самыми сложными вопросами:
5908. Лига Сибири. III тур.
5907. Осенняя кинолига
5109. Серия Гран-при. 6 этап. Гран-при Восточной Европы
6063. ВДИ - С Новым Годом!
5779. Intermezzo
5929. Мемориал памяти Михаила Басса
5568. Голова профессора Доуэля
5934. Кубок Слегка НеСредиземья
5407. Девятый круг
5924. Кубок Оливье


In [223]:
print('Турниры с самыми простыми вопросами:')
for idx in tour_compl[-10:]:
    print(f"{idx[0]}. {tournaments[idx[0]]['name']}")

Турниры с самыми простыми вопросами:
5948. Чемпионат Мира. Финал. Группа А
5685. Боровские соборы
5687. Кубок соседней галактики. Эпсилон
5985. Українська ліга. Етап 1
6173. Кубок Мэра Казани
5274. Умами
5686. Хороший, плохой, синхрон
5680. Зеркало Ревельской весны
6161. Студенческий чемпионат Тюменской области
6183. Асинхрон турнира "Год театра"
